In [ ]:
import xarray as xr

from clustering import *
from radially_constrained_cluster import *

In [ ]:
dataset_t2m = xr.open_mfdataset(r'C:\Users\ITJG104327\Documents\Venezia\Dataset\ERA5\2m_temperature/*.nc').resample(time='1D').mean().rolling(time=15).mean()
dataset_tp = xr.open_mfdataset(r'C:\Users\ITJG104327\Documents\Venezia\Dataset\ERA5\total_precipitation/*.nc').resample(time='1D').sum().rolling(time=15).mean()

In [ ]:
dataset_t2m = dataset_t2m.sel(latitude=slice(46, 45), longitude=slice(11, 12))
dataset_t2m = dataset_t2m.groupby('time.dayofyear').mean()

In [ ]:
dataset_tp = dataset_tp.sel(latitude=slice(46, 45), longitude=slice(11, 12))
dataset_tp = dataset_tp.groupby('time.dayofyear').mean()

In [ ]:
array_t2m = dataset_t2m.t2m.to_numpy()
array_t2m = np.reshape(array_t2m, (array_t2m.shape[0], array_t2m.shape[1]*array_t2m.shape[2]))

In [ ]:
array_tp = dataset_tp.tp.to_numpy()
array_tp = np.reshape(array_tp, (array_tp.shape[0], array_tp.shape[1]*array_tp.shape[2]))

In [ ]:
# concatenate the two arrays on columns
array_tot = np.concatenate((array_t2m, array_tp), axis=1)

In [ ]:
model = Radially_Constrained_Cluster(array_tot, 4, 400, 
                                1, True, 0.1, 
                                True, 20)

model.fit()
prediction = model.get_prediction()

fig_learning = plt.figure(figsize = (15,6))
plt.plot(model.error_story)
plt.grid()
plt.xlabel('N° iterations')
plt.ylabel('Within-Cluster Sum of Square')

In [ ]:
cc = model.get_centroids()  
bb = model.breakpoints

In [ ]:
fig_clustered_field, ax = plt.subplots(figsize = (15,6))

dataset_t2m.t2m.mean(['latitude', 'longitude']).plot(ax=ax)
for b in bb:
    ax.axvline(b, color='k', linestyle='--', linewidth=1)

ax.set_xticks([0,31,60,91,121,152,182,213,244,274,305,335,366],['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec','jan'])

fig_clustered_field, ax = plt.subplots(figsize = (15,6))

dataset_tp.tp.mean(['latitude', 'longitude']).plot(ax=ax)
for b in bb:
    ax.axvline(b, color='k', linestyle='--', linewidth=1)

ax.set_xticks([0,31,60,91,121,152,182,213,244,274,305,335,366],['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec','jan'])